# Load the Pretrained Model and the Dataset

In [1]:
import sys

sys.path.insert(0, "../..")
sys.path.insert(0, "../../../")

In [2]:
import paddle
import paddlenlp
from paddlenlp.transformers import ErnieForSequenceClassification, ErnieTokenizer

MODEL_NAME = 'ernie-2.0-en'

model = ErnieForSequenceClassification.from_pretrained(MODEL_NAME, num_classes=2)
tokenizer = ErnieTokenizer.from_pretrained(MODEL_NAME)

[2022-06-07 14:38:18,410] [    INFO] - Already cached /home/zhangshuai/.paddlenlp/models/ernie-2.0-en/ernie_v2_eng_base.pdparams
W0607 14:38:18.414284 31982 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.4, Runtime API Version: 10.2
W0607 14:38:18.420110 31982 device_context.cc:465] device: 0, cuDNN Version: 8.2.
[2022-06-07 14:38:23,873] [    INFO] - Already cached /home/zhangshuai/.paddlenlp/models/ernie-2.0-en/vocab.txt


In [3]:
from paddlenlp.datasets import load_dataset

DATASET_NAME = 'qqp'
train_ds, dev_ds, test_ds = load_dataset("glue", name='qqp', splits=["train", "dev", "test"])

# Prepare the Model
## Train the model

In [ ]:
from assets.utils import training_model

training_model(model, tokenizer, train_ds, dev_ds, save_dir=f'../../assets/{DATASET_NAME}-{MODEL_NAME}')

## Or Load the trained model

In [4]:
# Load the trained model.
!wget --no-check-certificate -c https://trustai.bj.bcebos.com/qqp-ernie-2.0-en.tar
!tar -xvf ./qqp-ernie-2.0-en.tar -C ../../assets/
!rm ./qqp-ernie-2.0-en.tar

state_dict = paddle.load(f'../../assets/{DATASET_NAME}-{MODEL_NAME}/model_state.pdparams')
model.set_dict(state_dict)

--2022-06-07 14:38:26--  https://trustai.bj.bcebos.com/qqp-ernie-2.0-en.tar
Resolving trustai.bj.bcebos.com (trustai.bj.bcebos.com)... 10.70.0.165
Connecting to trustai.bj.bcebos.com (trustai.bj.bcebos.com)|10.70.0.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 438200320 (418M) [application/x-tar]
Saving to: ‘qqp-ernie-2.0-en.tar’

100%[======================================>] 438,200,320  124MB/s   in 3.7s   

2022-06-07 14:38:29 (112 MB/s) - ‘qqp-ernie-2.0-en.tar’ saved [438200320/438200320]

qqp-ernie-2.0-en/
qqp-ernie-2.0-en/tokenizer_config.json
qqp-ernie-2.0-en/vocab.txt
qqp-ernie-2.0-en/model_state.pdparams
qqp-ernie-2.0-en/model_config.json


# See the prediciton results

In [5]:
from assets.utils import predict

data = [
    {'sentence1': 'What are the best novels in 2016?', 'sentence2': 'What are some of the best novels everyone should read?', 'labels': 0},
    {'sentence1': 'How can I get rid of stage fear?', 'sentence2': 'How do I get rid of my stage fear?', 'labels': 1},
    {'sentence1': 'Is it illegal to use iTunes music in a video?', 'sentence2': 'Can anyone get the music used in this video?', 'labels': 0},
]

label_map = {0: 'negative', 1: 'positive'}

batch_size = 32

results = predict(
    model, data, tokenizer, label_map, batch_size=batch_size)

for idx, text in enumerate(data):
    print('Data: {} \t Lable: {}'.format(text, results[idx]))

Data: {'sentence1': 'What are the best novels in 2016?', 'sentence2': 'What are some of the best novels everyone should read?', 'labels': 0} 	 Lable: negative
Data: {'sentence1': 'How can I get rid of stage fear?', 'sentence2': 'How do I get rid of my stage fear?', 'labels': 1} 	 Lable: positive
Data: {'sentence1': 'Is it illegal to use iTunes music in a video?', 'sentence2': 'Can anyone get the music used in this video?', 'labels': 0} 	 Lable: negative


# Prepare for Interpretations

In [6]:
from functools import partial

from paddlenlp.data import Stack, Tuple, Pad

from assets.utils import create_dataloader, convert_example

DATASET_NAME = 'qqp'

batch_size = 32
max_seq_length = 128
learning_rate = 5e-5 
epochs = 3
warmup_proportion = 0.1
weight_decay = 0.01

trans_func = partial(
        convert_example,
        tokenizer=tokenizer,
        max_seq_length=max_seq_length,
        is_test=True,
    )
batchify_fn = lambda samples, fn=Tuple(
    Pad(axis=0, pad_val=tokenizer.pad_token_id),  # input
    Pad(axis=0, pad_val=tokenizer.pad_token_type_id),  # segment
): [data for data in fn(samples)]
train_data_loader = create_dataloader(
    train_ds,
    mode='train',
    batch_size=batch_size,
    batchify_fn=batchify_fn,
    trans_fn=trans_func,
    shuffle=False)

# Rrepresenter Pointer Interpreter

In [7]:
from trustai.interpretation.example_level.method.representer_point import RepresenterPointModel

# classifier_layer_name is the layer name of the last output layer
representer_model = RepresenterPointModel(model, train_data_loader, classifier_layer_name='classifier')

Extracting feature for given dataloader, it will take some time...
Training representer point model, it will take several minutes...
L1 difference between ground truth prediction and prediction by representer theorem decomposition
[0.00103879]
pearson correlation between ground truth  prediction and prediciton by representer theorem
0.9999959119461196


In [8]:
from assets.utils import create_dataloader_from_scratch, print_result

test_data = [
    {'sentence1': 'What are the best novels in 2016?', 'sentence2': 'What are some of the best novels everyone should read?', 'labels': 0},
    {'sentence1': 'How can I get rid of stage fear?', 'sentence2': 'How do I get rid of my stage fear?', 'labels': 1},
    {'sentence1': 'Is it illegal to use iTunes music in a video?', 'sentence2': 'Can anyone get the music used in this video?', 'labels': 0},
]

# process text to model input
test_dataloader = create_dataloader_from_scratch(test_data, tokenizer)

res = representer_model.interpret(test_dataloader)

print_result(test_data, train_ds, res, data_name='qqp')

Extracting feature for given dataloader, it will take some time...
test data
text: What are the best novels in 2016?	What are some of the best novels everyone should read?	predict label: 0
examples with positive influence
text: How do I sell domains?	What's the best way to sell a domain?	gold label: 0	score: 0.0002862816909328103
text: How do I make chocolate brownies?	What's the best chocolate brownie recipe?	gold label: 0	score: 0.00023215859255287796
text: How do I lose weight faster?	What are the best ways to lose weight?	gold label: 0	score: 0.00022417452419176698
examples with negative influence
text: What are some query languages?sepsepWhat is query language?	gold label: 1	score: -0.00017490192840341479
text: What should I do if girl is sending mixed signals?sepsepWhy do girls give mixed signals?	gold label: 0	score: -0.0001809245441108942
text: Can we time travel anyhow?sepsepWhy can't we time travel?	gold label: 0	score: -0.00020212694653309882

test data
text: How can I get r